# Création d'un nouveau dataset

seance_projet_20190319_new_columns

Import des packages utilisés

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Import des fichiers nécessaires

In [2]:
#fichiers réduits des transactions avec les card_id du train.csv
hist_train = pd.read_csv("hist_mini_20190221.csv", index_col = 0)
new_train = pd.read_csv("new_mini_20190221.csv", index_col = 0)

#fichiers réduits des transactions avec les card_id du test.csv
hist_test = pd.read_csv("hist_mini_test_20190222.csv", index_col = 0)
new_test = pd.read_csv("new_mini_test_20190222.csv", index_col = 0)

#Les datasets train et test
df_train = pd.read_csv("train.csv", index_col = 1) # indexé par card_id
df_test = pd.read_csv("test.csv", index_col = 1) # indexé par card_id

C:\OUTILS\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Récupération des card_id du train et du test

In [3]:
card_id_train = list(pd.read_csv("train.csv", usecols = ["card_id"], squeeze = True))
card_id_test = list(pd.read_csv("test.csv", usecols = ["card_id"], squeeze = True))

In [5]:
hist_train.head()

,card_id,month_lag,purchase_amount,purchase_date,subsector_id
29100000,C_ID_732dcdbb46,0,0.282408,2018-02-17 08:35:10,2
29100001,C_ID_732dcdbb46,-4,0.090174,2017-10-20 15:44:57,1
29100002,C_ID_69082a8187,-2,-0.300711,2017-11-02 12:56:54,16
29100003,C_ID_69082a8187,-10,0.346436,2017-03-06 18:20:59,27
29100004,C_ID_69082a8187,-10,-0.629701,2017-03-06 16:16:10,29


In [6]:
new_train.head()

,card_id,month_lag,purchase_amount,purchase_date,subsector_id
1900010,C_ID_7e76630927,1,-0.722880,2018-03-14 16:46:07,1
1900014,C_ID_8d2802657c,2,-0.596643,2018-04-21 20:54:29,41
1900015,C_ID_8d2802657c,2,-0.735638,2018-04-11 07:49:58,37
1900016,C_ID_8d2802657c,1,-0.724368,2018-03-28 19:39:17,19
1900017,C_ID_9330917d31,2,-0.221220,2018-04-03 11:58:35,27


Datasets train et test : ajout d'une colonne active_months, qui est un nombre de mois

In [7]:
df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target
card_id,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283
C_ID_3d0044924f,2017-01,4,1,0,0.392913
C_ID_d639edf6cd,2016-08,2,2,0,0.688056
C_ID_186d6a6901,2017-09,4,3,0,0.142495
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749


In [8]:
df_train["first_active_month"].sort_values(ascending = False)[:2]


card_id
C_ID_668b351e9f    2018-02
C_ID_74204306cd    2018-01
Name: first_active_month, dtype: object

In [62]:
nb_mois = (12 * (2018 - df_train["first_active_month"].apply(lambda x: int(x[:4]))) + 2 - df_train["first_active_month"].apply(lambda x: int(x[5:7])))
df_train["active_months"] = nb_mois
df_train = df_train.drop("first_active_month", axis = 1)
df_train.head()

,feature_1,feature_2,feature_3,target,nb_purchases_hist,nb_purchases_new,purchase_amount_tot_hist,purchase_amount_tot_new,active_months
card_id,,,,,,,,,
C_ID_92a2005557,5,2,1,-0.820283,260,23.0,-165.968739,-13.244202,8
C_ID_3d0044924f,4,1,0,0.392913,350,6.0,-210.006336,-4.355735,13
C_ID_d639edf6cd,2,2,0,0.688056,43,1.0,-29.167391,-0.700326,18
C_ID_186d6a6901,4,3,0,0.142495,77,7.0,-49.491364,-4.654372,5
C_ID_cdbd2c0db2,1,3,0,-0.159749,133,36.0,-48.687656,-19.926237,3


Cas du dataset test : une valeur valeur manquante pour le first_active month, on remplace par "2017-09" qui apparait le plus souvent

In [10]:
pd.isnull(df_test).sum()

first_active_month    1
feature_1             0
feature_2             0
feature_3             0
dtype: int64

In [63]:
#Une valeur manquante pour le first_active month, on remplace par "2017-09" qui apparait le plus
#df2  = df_test.dropna()
#nb_mois = (12 * (2018 - df2["first_active_month"].apply(lambda x: int(x[:4]))) + 2 - df2["first_active_month"].apply(lambda x: int(x[5:7])))
#nb_mois.value_counts()[:3]
df_test["first_active_month"] = df_test["first_active_month"].fillna("2017-09")
nb_mois = (12 * (2018 - df_test["first_active_month"].apply(lambda x: int(x[:4]))) + 2 - df_test["first_active_month"].apply(lambda x: int(x[5:7])))

df_test["active_months"] = nb_mois
df_test = df_test.drop("first_active_month", axis = 1)
df_test.head()

,feature_1,feature_2,feature_3,active_months
card_id,,,,
C_ID_0ab67a22ab,3,3,1,10
C_ID_130fd0cbdd,2,3,0,13
C_ID_b709037bc5,5,1,1,6
C_ID_d27d835a9f,2,1,0,2
C_ID_2b5e3df5c2,5,1,1,26


### Ajout des colonnes de transactions en séparant les résultats pour hist et new

Nombre d'achats

In [35]:
# Fichier train
nb_achats_hist = hist_train["card_id"].value_counts()
nb_achats_new = new_train["card_id"].value_counts()

nb_achats_hist = nb_achats_hist.rename("nb_purchases_hist")
nb_achats_new = nb_achats_new.rename("nb_purchases_new")

df_train = df_train.join(nb_achats_hist, on = "card_id", how = "outer")
df_train = df_train.join(nb_achats_new, on = "card_id", how = "outer")

df_train["nb_purchases_new"].fillna(0, inplace = True) # parce que toutes les card_id ne sont pas dans new


In [36]:
df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,nb_purchases_hist,nb_purchases_new
card_id,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,260,23.0
C_ID_3d0044924f,2017-01,4,1,0,0.392913,350,6.0
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,43,1.0
C_ID_186d6a6901,2017-09,4,3,0,0.142495,77,7.0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,133,36.0


In [37]:
pd.isnull(df_train).sum()

first_active_month    0
feature_1             0
feature_2             0
feature_3             0
target                0
nb_purchases_hist     0
nb_purchases_new      0
dtype: int64

In [64]:
# Fichier test
nb_achats_hist = hist_test["card_id"].value_counts()
nb_achats_new = new_test["card_id"].value_counts()

nb_achats_hist = nb_achats_hist.rename("nb_purchases_hist")
nb_achats_new = nb_achats_new.rename("nb_purchases_new")

df_test = df_test.join(nb_achats_hist, on = "card_id", how = "outer")
df_test = df_test.join(nb_achats_new, on = "card_id", how = "outer")

df_test["nb_purchases_new"].fillna(0, inplace = True) # parce que toutes les card_id ne sont pas dans new


In [65]:
pd.isnull(df_test).sum()

feature_1            0
feature_2            0
feature_3            0
active_months        0
nb_purchases_hist    0
nb_purchases_new     0
dtype: int64

Montant des achats

In [43]:
#Fichier train
purchase_amount_tot_hist = hist_train.groupby("card_id").sum()["purchase_amount"]
purchase_amount_tot_new = new_train.groupby("card_id").sum()["purchase_amount"]

purchase_amount_tot_hist = purchase_amount_tot_hist.rename("purchase_amount_tot_hist")
purchase_amount_tot_new = purchase_amount_tot_new.rename("purchase_amount_tot_new")

df_train = df_train.join(purchase_amount_tot_hist, on = "card_id", how = "outer")
df_train = df_train.join(purchase_amount_tot_new, on = "card_id", how = "outer")

df_train["purchase_amount_tot_new"].fillna(0, inplace = True) # parce que toutes les card_id ne sont pas dans new


In [44]:
df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,nb_purchases_hist,nb_purchases_new,purchase_amount_tot_hist,purchase_amount_tot_new
card_id,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,260,23.0,-165.968739,-13.244202
C_ID_3d0044924f,2017-01,4,1,0,0.392913,350,6.0,-210.006336,-4.355735
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,43,1.0,-29.167391,-0.700326
C_ID_186d6a6901,2017-09,4,3,0,0.142495,77,7.0,-49.491364,-4.654372
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,133,36.0,-48.687656,-19.926237


In [45]:
pd.isnull(df_train).sum()

first_active_month          0
feature_1                   0
feature_2                   0
feature_3                   0
target                      0
nb_purchases_hist           0
nb_purchases_new            0
purchase_amount_tot_hist    0
purchase_amount_tot_new     0
dtype: int64

In [66]:
purchase_amount_tot_hist = hist_test.groupby("card_id").sum()["purchase_amount"]
purchase_amount_tot_new = new_test.groupby("card_id").sum()["purchase_amount"]

purchase_amount_tot_hist = purchase_amount_tot_hist.rename("purchase_amount_tot_hist")
purchase_amount_tot_new = purchase_amount_tot_new.rename("purchase_amount_tot_new")

df_test = df_test.join(purchase_amount_tot_hist, on = "card_id", how = "outer")
df_test = df_test.join(purchase_amount_tot_new, on = "card_id", how = "outer")

df_test["purchase_amount_tot_new"].fillna(0, inplace = True) # parce que toutes les card_id ne sont pas dans new

In [69]:
pd.isnull(df_test).sum()

feature_1                   0
feature_2                   0
feature_3                   0
active_months               0
nb_purchases_hist           0
nb_purchases_new            0
purchase_amount_tot_hist    0
purchase_amount_tot_new     0
dtype: int64

Date du dernier achat

In [ ]:
#Fichier train
last_purchase_hist = hist_train.groupby("card_id").max()["purchase_date"]
last_purchase_new = new_train.groupby("card_id").max()["purchase_date"]

last_purchase_hist = last_purchase_hist.rename("last_purchase_hist")
last_purchase_new = last_purchase_new.rename("last_purchase_new")

df_train = df_train.join(last_purchase_hist, on = "card_id", how = "outer")
df_train = df_train.join(last_purchase_new, on = "card_id", how = "outer")

df_train["last_purchase_new"].fillna(0, inplace = True) # parce que toutes les card_id ne sont pas dans new


In [71]:
pd.isnull(df_train).sum()

feature_1                   0
feature_2                   0
feature_3                   0
target                      0
nb_purchases_hist           0
nb_purchases_new            0
purchase_amount_tot_hist    0
purchase_amount_tot_new     0
active_months               0
last_purchase_hist          0
last_purchase_new           0
dtype: int64

In [72]:
df_train.head()

,feature_1,feature_2,feature_3,target,nb_purchases_hist,nb_purchases_new,purchase_amount_tot_hist,purchase_amount_tot_new,active_months,last_purchase_hist,last_purchase_new
card_id,,,,,,,,,,,
C_ID_92a2005557,5,2,1,-0.820283,260,23.0,-165.968739,-13.244202,8,2018-02-25 09:31:15,2018-04-29 11:23:05
C_ID_3d0044924f,4,1,0,0.392913,350,6.0,-210.006336,-4.355735,13,2018-01-31 22:31:09,2018-03-30 06:48:26
C_ID_d639edf6cd,2,2,0,0.688056,43,1.0,-29.167391,-0.700326,18,2018-02-27 19:08:25,2018-04-28 17:43:11
C_ID_186d6a6901,4,3,0,0.142495,77,7.0,-49.491364,-4.654372,5,2018-02-28 11:44:40,2018-04-18 11:00:11
C_ID_cdbd2c0db2,1,3,0,-0.159749,133,36.0,-48.687656,-19.926237,3,2018-02-28 20:40:41,2018-04-28 18:50:25


In [74]:
#Fichier test
last_purchase_hist = hist_test.groupby("card_id").max()["purchase_date"]
last_purchase_new = new_test.groupby("card_id").max()["purchase_date"]

last_purchase_hist = last_purchase_hist.rename("last_purchase_hist")
last_purchase_new = last_purchase_new.rename("last_purchase_new")

df_test = df_test.join(last_purchase_hist, on = "card_id", how = "outer")
df_test = df_test.join(last_purchase_new, on = "card_id", how = "outer")

df_test["last_purchase_new"].fillna(0, inplace = True) # parce que toutes les card_id ne sont pas dans new

In [75]:
pd.isnull(df_test).sum()

feature_1                   0
feature_2                   0
feature_3                   0
active_months               0
nb_purchases_hist           0
nb_purchases_new            0
purchase_amount_tot_hist    0
purchase_amount_tot_new     0
last_purchase_hist          0
last_purchase_new           0
dtype: int64

In [76]:
df_test.head()

,feature_1,feature_2,feature_3,active_months,nb_purchases_hist,nb_purchases_new,purchase_amount_tot_hist,purchase_amount_tot_new,last_purchase_hist,last_purchase_new
card_id,,,,,,,,,,
C_ID_0ab67a22ab,3,3,1,10,68,3.0,-40.733733,-1.777156,2017-12-29 01:05:24,2018-02-28 19:08:52
C_ID_130fd0cbdd,2,3,0,13,78,9.0,-49.136513,-5.944698,2018-02-18 21:30:11,2018-04-20 18:00:17
C_ID_b709037bc5,5,1,1,6,13,2.0,4.528840,0.180138,2018-02-02 18:57:03,2018-03-13 13:10:46
C_ID_d27d835a9f,2,1,0,2,26,10.0,-13.690715,-5.743674,2018-02-20 11:42:39,2018-04-17 21:22:23
C_ID_2b5e3df5c2,5,1,1,26,110,6.0,25.139384,12.064997,2018-02-27 10:42:04,2018-04-12 12:13:11


In [77]:
df_train.to_csv("df_train20190319.csv")
df_test.to_csv("df_test20190319.csv")